## DCGAN 生成人脸
在这个项目中，我们会使用前面学到的深度卷积生成对抗网络来生成人脸，使用 CalebA 人脸数据集，一共有202599张人脸图片。

首先我们导入需要的库

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

import torchvision.transforms as tfs
from torch.utils.data import DataLoader, sampler
from torchvision.datasets import ImageFolder

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from utils import show_images, preprocess_img, deprocess_img

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 设置画图的尺寸
plt.rcParams['image.interpolation'] = 'nearest'

下面我们可视化一张图片看看

In [ ]:
from PIL import Image
im = Image.open('./dataset/img_align_celeba/000001.jpg')
im

可以看到在这张图片上，人脸只占中心的部分，所以我们会做一个预处理，取出图片中心的人脸图像，然后 resize 到 (28, 28) 的大小

下面我们取出一个 batch_size 的图像进行可视化

In [ ]:
# 可以调整的超参数
batch_size = 32
use_gpu = True
NOISE_DIM = 96
epochs = 30

# ========不要修改下面的部分========
train_set = ImageFolder('./dataset/', transform=preprocess_img)

train_data = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)


imgs = deprocess_img(iter(train_data).next()[0].numpy()) # 可视化图片效果
show_images(imgs)

## 建立模型

### 构建判别网络
首先建立判别网络，由前面的课程我们知道判别网络就是一个二分类网络

In [ ]:
class build_dc_classifier(nn.Module):
    def __init__(self):
        super(build_dc_classifier, self).__init__()
        # todo: 建立判别网络

### 建立生成网络
下面我们利用线性层和反卷积来建立一个生成网络，可以查看一下前面的课程寻找一些灵感

In [ ]:
class build_dc_generator(nn.Module): 
    def __init__(self, noise_dim=NOISE_DIM):
        super(build_dc_generator, self).__init__()
        # todo: 建立生成网络

### 构建 loss
在前面的课程中我们已经描述并实现过生成对抗网络的loss，下面请不要查看前面的课程，自己再实现一下

In [ ]:
bce_loss = nn.BCEWithLogitsLoss()

def discriminator_loss(logits_real, logits_fake):
    # todo: 完成判别器的 loss

In [ ]:
def generator_loss(logits_fake):
    # todo: 完成生成器的 loss

In [ ]:
def get_optimizer(net):
    # todo: 使用 adam 进行优化，自己尝试一些学习率

下面请根据提示完成生成对抗网络的训练过程

In [ ]:
def train_dc_gan(D_net, G_net, D_optimizer, G_optimizer, discriminator_loss, generator_loss, show_every=250, 
                noise_size=NOISE_DIM, num_epochs=10):
    iter_count = 0
    for epoch in range(num_epochs):
        for x, _ in train_data:
            bs = x.shape[0]
            if use_gpu:
                x = x.cuda()
                
            # ======== 判别网络 =========
            # 真实数据
            real_data = Variable(x)
            # todo: 判别网络判断真实数据得分
            
            sample_noise = None # todo: 生成 -1 ~ 1 的均匀分布噪声数据，形状是 (bs, NOISE_DIM)
            if use_gpu:
                sample_noise = sample_noise.cuda()
            g_fake_seed = Variable(sample_noise)
            
            fake_images = None # todo: 通过生成网络生成的假的数据
            logits_fake = None # todo: 判别网络判断假的数据得分

            d_total_error = None #todo: 判别器的 loss
            
            # todo: 反向传播，更新判别网络的参数
    
            
            # ======== 生成网络 ==========
            g_fake_seed = Variable(sample_noise)
            fake_images = # todo: 通过生成网络生成的假的数据

            gen_logits_fake = # todo: 判别网络判断假的数据得分
            g_error = #todo: 得到生成器的 loss
            
            # todo: 反向传播，更新生成网络的参数

            if (iter_count % show_every == 0):
                print('Iter: {}, D: {:.4}, G:{:.4}'.format(iter_count, d_total_error.data[0], g_error.data[0]))
                imgs_numpy = deprocess_img(fake_images.data.cpu().numpy())
                show_images(imgs_numpy[0:16])
                plt.show()
                print()
            iter_count += 1

定义完了训练生成对抗网络的过程，我们下面进行训练，看看能否生成比较像真人的人脸

In [ ]:
D_DC = build_dc_classifier()
G_DC = build_dc_generator()

if use_gpu:
    D_DC = D_DC.cuda()
    G_DC = G_DC.cuda()

D_DC_optim = get_optimizer(D_DC)
G_DC_optim = get_optimizer(G_DC)

train_dc_gan(D_DC, G_DC, D_DC_optim, G_DC_optim, discriminator_loss, generator_loss, num_epochs=epochs)